In [21]:
from bs4 import BeautifulSoup 
import re 
import requests
import os
import nltk 
from sklearn import feature_extraction
from gensim import models
import matplotlib
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Embedding, Bidirectional
from keras.callbacks import ModelCheckpoint, LambdaCallback
import numpy as np

In [ ]:
# Faz uma requisição na página que contém mil links de letras de músicas sertanejas
url_artistas = "https://www.vagalume.com.br/browse/style/sertanejo.html"
pagina = requests.get(url_artistas)

In [ ]:
# Guarda o conteúdo da página
conteudo = pagina.content

In [ ]:
# Cria o parser HTML
soup = BeautifulSoup(conteudo,"html.parser")

In [ ]:
colunas_artistas = soup.findAll("ul",attrs={"class" : "namesColumn"})

In [ ]:
links_artistas = []
for c in colunas_artistas:
    links_artistas.extend(c.findAll("a"))

In [ ]:
len(links_artistas)

In [ ]:
links_artistas[0]

In [ ]:
raiz_site = "https://www.vagalume.com.br/"
links_musicas = []
for link in links_artistas:
    pagina_artista = requests.get(raiz_site + link.get("href"))
    soup_artista = BeautifulSoup(pagina_artista.content,"html.parser")
    div_musicas = soup_artista.find("ol", attrs={"id" : "alfabetMusicList"})
    if div_musicas is None:
        continue
        
    musicas_artistas = div_musicas.findAll("a",attrs={"class","nameMusic"})
    
    if musicas_artistas is None:
        continue
        
    print("Encontradas {} musicas para o artista {}".format(len(musicas_artistas),link.get("href")))
    for m in musicas_artistas:
        links_musicas.append(m.get("href"))
    
#     nome_arquivo = str(link.get("href")).replace("/","") + ".html"
#     with open("dados/letras-musicas/" + nome_arquivo,"w") as f:
#         f.write(str(artigo_letra))

In [ ]:
len(links_musicas)

In [ ]:
for link in links_musicas:
    pagina_musica = requests.get(raiz_site + link)
    soup_musica = BeautifulSoup(pagina_musica.content,"html.parser")
    artigo_letra = soup_musica.find("div",attrs={"id" : "lyrics"})
    nome_arquivo = link.replace("/","")
#     print(nome_arquivo,str(artigo_letra))
    with open("dados/letras-musicas/vagalume/sertanejo/" + nome_arquivo,"w") as f:
        f.write(str(artigo_letra))

In [2]:
documentos = []
diretorio = "dados/letras-musicas/vagalume/sertanejo/"
arquivos = os.listdir(diretorio)
for arquivo in arquivos:
    caminho = diretorio + arquivo
    if os.path.isfile(caminho):
        with open(caminho,"r") as f:
            html = f.read()
        html = feature_extraction.text.strip_accents_ascii(html.lower())
        html = html.replace("</br>","")
        html = html.replace("<br>"," NLINHA ")
        html = html.replace("<p>"," NLINHA ")
        soup = BeautifulSoup(html,"html.parser")
        documentos.append(soup.text.strip())

In [3]:
docs_tokenizados = []
tokenizador = nltk.TweetTokenizer()
for doc in documentos:
    tokens = tokenizador.tokenize(doc)
    docs_tokenizados.append(tokens)

In [4]:
palavras = []
for doc in docs_tokenizados:
    palavras.extend(doc)
unicas = set(palavras)
len(unicas)

47180

In [ ]:
documentos[5]

In [ ]:
docs_tokenizados[5]

In [5]:
# Treinando um modelo Word2vec nos documentos
w2v_model = models.Word2Vec(docs_tokenizados, size=350, window=15, min_count=7, workers=os.cpu_count(),iter=300)

In [6]:
w2v_model.wv["amor"]

array([ 4.5755592e-01,  1.0927660e+00,  8.0538118e-01,  3.6659946e+00,
       -1.5168151e+00, -2.6011224e+00,  4.9372632e-02,  1.5058969e-02,
        1.4393237e+00, -1.6073844e+00,  4.2827969e+00,  6.5023863e-01,
       -1.0618950e+00,  1.1108471e+00,  5.9222037e-01,  8.4944397e-01,
       -3.9825660e-01, -3.4656563e+00, -1.8381692e+00,  2.2230270e+00,
       -9.8075944e-01, -4.1292119e-01, -1.6673270e-01,  2.0306795e+00,
       -2.9277539e+00, -8.7580431e-01,  2.3321640e+00, -9.9248186e-02,
       -3.0750707e-01, -1.6168486e+00, -1.4469459e+00, -1.6511104e+00,
       -6.7717642e-01, -2.3463137e+00, -1.9644564e+00, -8.4327322e-01,
        1.8136808e+00, -3.4208217e-01, -1.1413827e+00, -5.8409777e-03,
        1.2573365e+00,  4.5235548e+00,  1.3686361e+00, -6.3972110e-01,
       -2.0832188e+00, -1.8590345e+00,  2.6616063e+00, -1.6003635e+00,
       -8.5423809e-01, -3.6808267e+00, -1.1638002e+00,  4.8066229e-01,
       -2.6829219e+00, -2.9357037e-01,  4.2329705e-01, -6.7765558e-01,
      

In [ ]:
# Cria um array com o vetor das duas mil primeiras palavras
vocab = list(w2v_model.wv.vocab)[:2000]
vetores_w2v = w2v_model.wv[vocab]

In [ ]:

vetores_w2v = np.asfarray(vetores_w2v)

In [ ]:
w2v_model.most_similar(positive=["."])

In [ ]:
# Reduzindo a dimensionalidade dos vetores das palavras
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
vetores_reduzidos = tsne.fit_transform(vetores_w2v)

In [ ]:
# Cria um dataframe com as colunas x e y. O objetivo é transformar cada representação de uma palavra em uma coordenada.
import pandas as pd
df = pd.DataFrame(vetores_reduzidos, index=vocab, columns=['x', 'y'])
df.head()

In [ ]:
# Utiliza o matplotlib com a chave de interação ligada.
%matplotlib notebook

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

for word, pos in df.iterrows():
    ax.annotate(word, pos) # Coloca a palavra em cada ponto

ax.scatter(df['x'], df['y']) # Cria um scatterPlot com todos os pontos x e y

In [7]:
w2v_model.save("modelos/w2v_sertanejo_28kmusicas_tweet_tknzr_window_15_mincount_7.model")

In [8]:
w2v_model.wv.vectors.shape

(14904, 350)

In [28]:
# Funcoes que retornam o índice da palavra no vocabulário criado pelo word2vec e vice-versa
def word2idx(word):
  return w2v_model.wv.vocab[word].index

def idx2word(idx):
  return w2v_model.wv.index2word[idx]

In [15]:
# Transforma as palavras que nao entraram no corte minimo de ocorrencias no corpus em um token UNKNOWN
vocab_w2v = list(w2v_model.wv.vocab)

counter=0
docs_tokens_w2v = []
for idx, doc in enumerate(docs_tokenizados[:100]):
    tokens_doc = []
    for token in doc:
        tokens_doc.append(token if token in w2v_model.wv.vocab else "UNK!")
    docs_tokens_w2v.append(tokens_doc)

In [17]:
# docs_tokenizados[:2]
docs_tokens_w2v[:2]

[['la',
  'em',
  'casa',
  'a',
  'festa',
  'nunca',
  'acaba',
  'NLINHA',
  'tudo',
  'termina',
  'em',
  'balada',
  'NLINHA',
  'sou',
  'professor',
  'de',
  'curticao',
  'NLINHA',
  'NLINHA',
  'la',
  'em',
  'casa',
  'tem',
  'sempre',
  'uns',
  'amigos',
  'NLINHA',
  'e',
  'qualquer',
  'coisa',
  'e',
  'motivo',
  'NLINHA',
  'para',
  'fazer',
  'UNK!',
  'NLINHA',
  'NLINHA',
  'la',
  'em',
  'casa',
  'a',
  'geladeira',
  'ta',
  'sempre',
  'cheia',
  'NLINHA',
  'a',
  'churrasqueira',
  'ta',
  'sempre',
  'acesa',
  'NLINHA',
  'e',
  'a',
  'gente',
  'ta',
  'sempre',
  'cantando',
  'NLINHA',
  'NLINHA',
  'la',
  'em',
  'casa',
  'e',
  'todo',
  'mundo',
  'meu',
  'convidado',
  'NLINHA',
  'gata',
  'dispensa',
  'seu',
  'namorado',
  'NLINHA',
  'e',
  'pode',
  'chegar',
  'chegando',
  'NLINHA',
  'NLINHA',
  'aqui',
  'quem',
  'faz',
  'a',
  'regras',
  'sou',
  'eu',
  'NLINHA',
  'se',
  'der',
  'b',
  '.',
  'o',
  'o',
  'problema',
  'e

In [22]:
vetor_unk = np.zeros(350, dtype=np.int32)
w2v_model.wv.add(["UNK!"],[vetor_unk])

In [26]:
maxlen = 15
sentences = []
next_tokens = []

for doc in docs_tokens_w2v: # Cria as "sentencas" para cada documento do corpus
    for i in range(0,len(doc) - maxlen):
        sentences.append(doc[i:i+maxlen])
        next_tokens.append(doc[i+maxlen])

print('Number of sequences:', len(sentences), "\n")

# sentences = sentences[0:30000] # Limitando em 30 mil sentenças para não demorar...
# next_tokens = next_tokens[0:30000]

Number of sequences: 17132 



In [29]:
# Criando os inputs(x) e targets(y)
x = np.zeros((len(sentences), maxlen), dtype=np.int32)
y = np.zeros((len(sentences)), dtype=np.int32)
for i, sentence in enumerate(sentences):
    for t, token in enumerate(sentence):
        x[i, t] = word2idx(token) 
    y[i] = word2idx(next_tokens[i])

In [30]:
pretrained_weights = w2v_model.wv.vectors

In [31]:
tamanho_vocab = pretrained_weights.shape[0]
tamanho_vetor_w2v = pretrained_weights.shape[1] # 350
print("Tamanho vocab e w2v vector: ", (tamanho_vocab, tamanho_vetor_w2v))

Tamanho vocab e w2v vector:  (14905, 350)


In [33]:
units1 = 60
units2 = 30
nome_modelo = "lstm-w2v-wordlevel-{}len-{}-{}-sertanejo-vagalume.model".format(maxlen, units1, units2)
caminho_modelo_lstm = "modelos/{}".format(nome_modelo)

In [34]:
model = Sequential()
model.add(Embedding(input_dim=tamanho_vocab, output_dim=tamanho_vetor_w2v, weights=[pretrained_weights]))
model.add(LSTM(units1, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=units2))
model.add(Dropout(0.1))
model.add(Dense(tamanho_vocab, activation='softmax'))  # Quantidade de 'respostas' possiveis. Tokens neste caso.
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

In [35]:
# define the checkpoint
checkpoint = ModelCheckpoint(caminho_modelo_lstm, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [37]:
def gerar_texto(epoch, logs):
    print('----- Generating text after Epoch: %d' % epoch)

    start = np.random.randint(0, len(sentences))
    seed_tokens = list(sentences[start])
    print("Seed:")
    print("\"", " ".join(seed_tokens), "\"\n")
    temperatura = 3

    aux = 1
    for i in range(500):

        xt = np.zeros((1, maxlen), dtype=np.int32)

        for t, token in enumerate(seed_tokens):
            xt[0, t] = word2idx(token)

        prediction = model.predict(xt, verbose=0)

        if temperatura > 0 and aux % temperatura == 0:
            ordered = (-prediction[0]).argsort()[:3]
            indice = np.random.choice(ordered)
        else:
            indice = (np.argmax(prediction))

        result = idx2word(indice)
        print("\n" if result == "NLINHA" else result, end=" ")
        seed_tokens.append(result)
        seed_tokens = seed_tokens[1:len(seed_tokens)]
        aux = aux + 1

    print("\n\nFIM\n\n", )

In [38]:
# Callback para gerar texto de exemplo ao final de cada iteração de treino
print_callback = LambdaCallback(on_epoch_end=gerar_texto)
callbacks_list.append(print_callback)

In [39]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 350)         5216750   
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 60)          98640     
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 60)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 30)                10920     
_________________________________________________________________
dropout_2 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 14905)             462055    
Total params: 5,788,365
Trainable params: 5,788,365
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.fit(x, y, epochs=30, batch_size=64, callbacks=callbacks_list)

Epoch 1/30
17132/17132 [==============================] - 28s 2ms/step - loss: 3.8204 - acc: 0.2722

Epoch 00001: loss improved from 3.85793 to 3.82042, saving model to modelos/lstm-w2v-wordlevel-15len-60-30-sertanejo-vagalume.model
----- Generating text after Epoch: 0
Seed:
" porta NLINHA mas o baile eu vou UNK! NLINHA mas deixa eu por a cabecinha "


 me dar que eu vou te dar 
 eu vou me apaixonar 
 mas eu vou me apaixonar 
 e eu vou me apaixonar 
 eu me quer 
 eu vou te dar 
 que eu vou te dar 
 eu vou te esquecer 
 e o amor nao me quer 
 , nao me quer 
 eu vou te quer que eu vou te apaixonar 
 voce nao te apaixonar 
 eu sou uma noticia pra te dar e a noticia com meu amor 
 eu vou me dar 
 voce nao te apaixonar 
 eu me quer 
 nao vou te apaixonar 
 eu me quer 
 eu vou te apaixonar 
 eu me quer 
 que nao te apaixonar 
 eu tenho uma noticia de seu amor de amor 
 pra amor de meu amor 
 e a noticia , me apaixonar que eu me apaixonar 
 eu nao me apaixonar e a noticia de seu amor de amor 

 que nao parar de seu amor 
 nao vou te apaixonar 
 ai , ai 
 ai , ai , ai , ai , mas eu vou me apaixonar 
 eu vou te apaixonar 
 eu me apaixonar 
 
 eu vou mais 
 voce e voce e meu amor da amor de amor 
 nao vou me apaixonar 
 que eu vou me apaixonar 
 eu vou te ver que eu te apaixonar 
 voce e voce , te apaixonar 
 eu vou mais te dar e a noticia 
 logo eu me apaixonar 
 
 eu vou me apaixonar 
 
 eu vou te dar 
 
 voce nao te apaixonar 
 
 nao vou te apaixonar 
 e eu vou mais me apaixonar e 
 eu sou que eu vou te apaixonar que eu vou te dar 
 eu vou te esquecer 
 
 sou uma noticia de meu amor da amor 
 nao vou te dar 
 eu parar , logo eu vou te dar 
 eu te apaixonar 
 que eu vou mais 
 voce e voce , me apaixonar 
 
 nao vou me apaixonar 
 eu vou te esquecer 
 
 eu vou te dar 
 que nao parar de amor de amor 
 e voce , nao vou 

FIM


Epoch 5/30
17132/17132 [==============================] - 36s 2ms/step - loss: 3.6540 - acc: 0.2853

Epoch 00005: loss improved from 3.69674 to 3.65404, s

 de UNK! UNK! com a UNK! de UNK! UNK! do meu UNK! , UNK! , e a noite e a noite e a vida 
 
 ta doidinha de UNK! de UNK! 
 na UNK! de sua UNK! de sua vida 
 a UNK! e a UNK! do corpo ) 
 que nao vou mais me quer e e eu quero te dar e 
 eu te apaixonar 
 e voce me quer 
 eu me apaixonar 
 e a moca e a noite , nao vou te dar 
 a moca nao me quer 
 e eu vou te apaixonar 
 e eu vou mais me apaixonar e 
 eu vou te dar 
 
 voce a noticia , voce me apaixonar 
 e a noite e a noite e UNK! com o UNK! de serra-sp molhadinha da UNK! 
 me UNK! 
 e a UNK! do UNK! nao me UNK! 
 eu sei e que nao ta UNK! de UNK! de UNK! e um UNK! de casa 
 flor . 
 
 se UNK! na terra de UNK! e UNK! de goias 
 
 e a noite e minha vida 
 
 sou facil de mim que nao quero te dar 
 eu vou te dar 
 eu vou te dar que eu vou te dar 
 que nao quer amor 
 voce me quer 
 voce me quer 
 
 ai , ai , esse amor 
 
 na moca e a moca nao me quer 
 
 eu me quer 
 pra eu me nao quer 
 , ai , esse amor 
 
 vou te esquecer 
 e uma vida 
 e a

 ai , ai , ai , ai , ai , ai , esse amor 
 eu vou te dar que nao parar , te dar 
 eu quero te dar 
 eu vou mais te dar que eu vou me apaixonar 
 eu vou te apaixonar 
 eu quero te dar e um amor 
 
 ai , ai , esse amor 
 eu vou peao 

FIM


Epoch 12/30
17132/17132 [==============================] - 35s 2ms/step - loss: 3.3804 - acc: 0.3171

Epoch 00012: loss improved from 3.43153 to 3.38037, saving model to modelos/lstm-w2v-wordlevel-15len-60-30-sertanejo-vagalume.model
----- Generating text after Epoch: 11
Seed:
" admirar NLINHA abri a janela pra poder ficar olhando NLINHA ela veio me empurrando : "

UNK! 
 no dia e no UNK! sonhos 
 na janela , UNK! a moca nao me faz 
 
 que eu vou me divertir 
 a moca nao nao me quer que eu nao vou me quer que eu tenho uma noticia , seu amor me quer 
 eu quero te dar que eu parar de seu pai que eu sou uma noticia , a noticia de meu amor 
 so me apaixonar e 
 eu me apaixonar 
 
 eu vou te dar 
 e voce nao quer 
 voce que nao parar de seu pai que eu quer

 voce da noite ? 
 e noite e UNK! de UNK! e UNK! 
 sobra em UNK! de dia 
 ta doidinha do cheiro de serra-sp paulo da serra-sp you UNK! . 
 UNK! do seu UNK! e UNK! 
 recebendo 
 UNK! de dia e as nuvens com o um UNK! 
 g UNK! 
 e que foi pra dar que eu vou mais me quer 
 
 eu vou te apaixonar , eu quero me dar 
 e voce me dar 
 eu tenho uma noticia , voce me apaixonar 
 e o meu coracao ? 
 
 vou me chamar , 
 mas voce nao quero me apaixonar 
 
 eu vou te dar 
 que eu vou mais me apaixonar que eu vou me apaixonar 
 eu tenho uma cabecinha 
 eu sou feliz 
 a noite inteira a noticia de seu pai que ta aqui , nunca te ver em amor . pra voce 
 sou o seu corpo da gente e um amor , so que foi em UNK! 
 flor , pra mim 
 quando eu vou me chamar 
 e proibido eu vou te dar que eu vou mais coracao 
 eu vou te chamar 
 
 vou te apaixonar e voce , sou que vou mais me quer e que eu quero te dar que eu vou mais mais em meu amor 
 voce me apaixonar que eu nao vou te dar e a noticia 
 logo nao vou te dar pr

17132/17132 [==============================] - 33s 2ms/step - loss: 3.1723 - acc: 0.3423

Epoch 00019: loss improved from 3.20020 to 3.17232, saving model to modelos/lstm-w2v-wordlevel-15len-60-30-sertanejo-vagalume.model
----- Generating text after Epoch: 18
Seed:
" logo eu vou me apaixonar NLINHA eu tenho uma noticia , uma noticia pra te "

dar 
 eu vou te apaixonar 
 eu tenho uma noticia , voce me dar 
 eu vou te dar 
 eu vou mais me apaixonar 
 eu vou te apaixonar 
 a cabecinha 
 voce nao quero mais 
 eu sou feliz 
 e voce me apaixonar 
 
 eu vou te esquecer 
 
 mas a moca nao nao nao vou me apaixonar que eu me me quer 
 
 eu vou contar de amor e 
 eu sei . 
 pra te dar 
 eu vou te apaixonar 
 a cabecinha 
 nao vou te ver mais pra voce e voce nao parar com meu coracao nao quero te esquecer 
 e so assim de dia 
 e na noticia de mim 
 eu quero me apaixonar 
 
 eu tenho outro meu coracao e a noticia pra te dar que eu vou te dar 
 que nao parar de mim 
 eu quero te esquecer . 
 que eu 

 so na vida . 
 
 a noite inteira e a vida . 
 
 ela ta doidinha do seu dia e na mulher e g do amor de casa , UNK! , e um dia e as nuvens assim , a vida 
 e o amor nao quero te esquecer 
 e uma noticia de mim , que nao vou te dar e um corpo da sua noticia pra te dar , meu amor que eu quero me dar 
 voce nao parar de mim 
 voce e uma mulher do amor nao oh , chamas te dar em amor nao sou uma noticia pra te dar e um amor e 
 e uma noticia , ela e paixao de seu coracao ! 
 e minha vida 
 se nao vou me apaixonar 
 eu tenho uma cabecinha 
 eu quero te dar que eu vou me apaixonar 
 a cabecinha que nao parar e um corpo nao quer entender eu te dar 
 eu quero te esquecer . 
 
 voce nao quero te dar amor 
 eu quero te dar e um corpo ) 
 pra dia do amor , sou em corpo de mim 
 mas voce nao quero ser um amor 
 e a noticia de mim 
 eu vou te encontrar , 
 logo eu quero te dar 
 que nao parar com isso , logo eu vou 

FIM


Epoch 23/30
17132/17132 [==============================] - 34s 2ms/step - loss

 ta gostando , ta querendo eu me quer 
 
 pois nao quero dar que eu parar de mim , eu sou o meu coracao ! 
 
 se nao parar , so a moca nao me faz 
 
 eu vou me apaixonar 
 eu tenho uma noticia 
 esse amor de amor 
 
 so nao sou feliz , a noite do seu coracao 
 
 vou contar de amor 
 pra te dar que eu parar e uma noite inteira 
 voce me quer bem de um UNK! 
 
 quando foi uma mulher que nao sei a cabecinha 
 meu amor . e nao quero mais mais . e 
 voce vai te dar que eu parar , sou uma mulher 
 e que nao quero te dar 
 nao vou te dar 
 eu podemos me apaixonar que eu vou te esquecer . que eu te ver mais mais que nao vou te dar 
 que nao parar de mim 
 eu sei que nao vou te esquecer 
 e tudo que podemos mais mais 
 que ce nao quero te esquecer 
 
 chora e a moca nao quer 
 que eu vou te esquecer 
 e a noite e minha vida 
 
 nao quero ser um dia que eu sei 
 
 logo nao vou contar pro seu pai que ce ta aqui 
 nao sabemos a sua boca 
 no saco que eu vou alcancar , 
 eu quero te dar que voce me

 e voce nao neste empurrando 
 e so me quer 
 com isso de seu lugar e pro meu UNK! 
 ela UNK! 
 mas e as nuvens ( UNK! de UNK! 
 ele da sua cama , 
 estou amarrado , leve as nuvens tem 
 ele eu tambem de UNK! e um molhadinha 
 UNK! 
 
 na janela , eu vou embora 
 eu tenho feliz 
 ai , ai , ai , ai , ai , ai , mas eu vou contar pro seu pai que ce me apaixonar 
 nao vou te apaixonar 
 eu tenho uma noticia , uma noticia , voce me ver mais a mim 
 voce nao quero te dar 
 e a mulher . 
 
 vou te dar 
 e uma noticia de meu amor 
 e seu amor de amor 
 de dia que eu vou embora 
 vou contar pro seu pai que ce me apaixonar 
 a gente termina na vida e que nao tem a gente termina de dia 
 ja ta doidinha de seus pes . 
 
 mas ta delirando e a UNK! de UNK! 
 ele da sua cama 
 e as ondas mais UNK! 
 
 

FIM


Epoch 30/30
17132/17132 [==============================] - 39s 2ms/step - loss: 2.8661 - acc: 0.3821

Epoch 00030: loss improved from 2.88594 to 2.86608, saving model to modelos/lstm-w2v-wordlev

In [41]:
gerar_texto(1,"teste")

----- Generating text after Epoch: 1
Seed:
" um desejo , NLINHA e a saudade que aperta o meu coracao . NLINHA NLINHA "


 
 
 
 
 
 
 
 e 
 
 , 
 
 e 
 
 , 
 
 
 
 
 , 
 
 , 
 
 , 
 
 
 
 
 e 
 
 , 
 
 , 
 
 , 
 
 , 
 
 , 
 
 , 
 
 , 
 
 
 
 
 , 
 
 , 
 
 
 
 
 e 
 
 e 
 
 , 
 
 , 
 
 , 
 
 , 
 
 , 
 
 
 
 
 
 
 
 e 
 
 e 
 
 , 
 
 , 
 
 , 
 
 , 
 
 e 
 
 
 
 
 e 
 
 , 
 
 , 
 
 , 
 
 
 
 
 
 
 
 , 
 
 , 
 
 
 
 
 e 
 
 , 
 
 , 
 
 
 
 
 e 
 
 
 
 
 
 
 
 e 
 
 , 
 
 
 
 
 , 
 
 
 
 
 , 
 
 , 
 
 , 
 
 , 
 
 
 
 
 , 
 
 
 
 
 e 
 
 
 
 
 e 
 
 
 
 
 , 
 
 e 
 
 , 
 
 e 
 
 e 
 
 , 
 
 e 
 
 
 
 
 , 
 
 
 
 
 
 
 
 
 
 
 , 
 
 e 
 
 , 
 
 , 
 
 e 
 
 e 
 
 
 
 
 e 
 
 e 
 
 
 
 
 
 
 
 
 
 
 e 
 
 e 
 
 e 
 
 , 
 
 
 
 
 e 
 
 
 
 
 
 
 
 e 
 
 e 
 
 e 
 
 , 
 
 
 
 
 e 
 
 e 
 
 e 
 
 e 
 
 , 
 
 
 
 
 , 
 
 e 
 
 e 
 
 
 
 
 , 
 
 
 
 
 , 
 
 , 
 
 
 
 
 
 
 
 
 
 
 , 
 
 e 
 
 
 
 
 e 
 
 
 
 
 e 
 
 , 
 
 e 
 
 
 
 
 e 
 
 
 
 
 
 
 
 , 
 
 , 
 
 e 
 
 , 
 
 
 
 
 
 
 
 , 
 
 e 
 
 